In [2]:
%load_ext autoreload
%autoreload 2
import startup_credit_2 as src_2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\Romain\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
RAW_CSV      = r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\ExternalData\startup_failures.csv"         # your Crunchbase-style file
MACRO_FOLDER = "ExternalData/"             # contains inflation.csv, gdp_growth.csv, unemployment_rate.csv
OUTPUT_PARQUET = "scored.parquet"
cfg = src_2.DEFAULT_CFG.copy()
cfg["momentum_variant"] = "auto"           # or "span" / "freq" / "recency"
cfg["stress_scenarios"]["mild"] = {"country_score_add": 0.5}

pl_df=src_2.load_and_clean(RAW_CSV)

In [5]:
df_comp = src_2.enrich_and_score(pl_df, learn_w=True, stress="baseline")

momentum-scores: 100%|██████████| 66368/66368 [00:26<00:00, 2478.47it/s]


In [129]:
df_comp.head()

,permalink,name,homepage_url,category_list,funding_total_usd,country_code,state_code,region,city,funding_rounds,...,score_momentum_avg_round,score_momentum_per_year,score_age,score_diversification,score_inflation,score_gdp_growth,score_unemployment_rate,score_total,rating,pd_model
0,/organization/-fame,#fame,http://livfame.com,[Media],10000000.0,IND,16,Mumbai,Mumbai,1,...,1.0,3.0,1,5,5,1,5,6.587360,R5,0.35
1,/organization/-qounter,:Qounter,http://www.qounter.com,"[Application Platforms, Real Time, Social Netw...",700000.0,USA,DE,DE - Other,Delaware City,2,...,5.0,7.0,1,3,3,1,3,5.614708,R4,0.20
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,"[Apps, Games, Mobile]",3406878.0,JPN,None,None,None,1,...,3.0,5.0,1,3,1,3,1,6.732206,R5,0.35
3,/organization/0-6-com,0-6.com,http://www.0-6.com,[Curated Web],2000000.0,CHN,22,Beijing,Beijing,1,...,3.0,7.0,1,5,1,3,1,7.483330,R5,0.35
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,[Software],3200000.0,USA,IL,"Springfield, Illinois",Champaign,1,...,3.0,5.0,1,5,3,1,3,6.326976,R4,0.20


In [ ]:
policy = src_2.CreditPolicy.from_training(
            df=df_comp[df_comp["target_binary"].notna()],
            comp_cols=[c for c in df_comp.columns if c.startswith(
                ("funding_total_usd", "diff_", "years_since_", "country_rate",
                "sector_rate","inflation", "gdp_growth", "unemployment_rate")
            )],
        )
#print(policy)
policy.to_yaml("policy_latest.yaml")
df_scored = policy.apply(df_comp, scenario="severe")
df_scored.head(10)

Deriving quantile bands...
Rating cutoffs: [-2.523685711022103, 0.5697205670588253, 1.1586209665291574, 1.6927172613391437]
4 5


In [ ]:
import os
import json
import pycountry
import polars as pl

def join_dfs(df) -> tuple:
    """
    Process CSV files in the given folder by:
      1. Converting specified year columns: casting to string, replacing "no data" with null,
         casting to Float64, and filling nulls with the 25th percentile.
      2. Renaming the first column to 'country' if needed.
      3. Extracting the text before a comma in the 'country' column and mapping it
         via countries_mapping to get a new 'country_code' column.
      4. Assuming there are three CSVs (unemployment, GDP per capita, inflation) in the folder,
         each DataFrame is further processed to melt the data to a long format and impute missing
         values using the mean of bordering countries.
    
    Parameters:

      neighbors_json_path (str): Path to a JSON file mapping countries to their neighbors.
      
    Returns:
      Tuple of Polars DataFrames: (unempl_filled, gdp_filled, infl_filled)
    """
    databank = pl.read_excel(r"ExternalData\P_Data_Extract_From_World_Development_Indicators.xlsx")

    unempl=databank.filter(pl.col("Series Code")=="SL.UEM.TOTL.NE.ZS").drop("Series Code","Series Name", "Country Code")
    ease_of_business=databank.filter(pl.col("Series Code")=="IC.BUS.DFRN.XQ").drop("Series Code","Series Name", "Country Code")
    cost_business=databank.filter(pl.col("Series Code")=="IC.REG.COST.PC.ZS").drop("Series Code","Series Name", "Country Code")
    r_d=databank.filter(pl.col("Series Code")=="IC.FRM.RSDV.ZS").drop("Series Code","Series Name", "Country Code")
    formal_training=databank.filter(pl.col("Series Code")=="IC.FRM.TRNG.ZS").drop("Series Code","Series Name", "Country Code")
    theft=databank.filter(pl.col("Series Code")=="IC.FRM.THEV.ZS").drop("Series Code","Series Name", "Country Code")
    market_cap=pl.read_csv(r"ExternalData\API_CM.MKT.LCAP.CD_DS2_en_csv_v2_18160.csv",skip_rows=4).drop("Indicator Name","Indicator Name", "Country Code")
    urban=pl.read_csv(r"ExternalData\API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_13366.csv",skip_rows=4).drop("Indicator Name","Indicator Name", "Country Code")
    lend=pl.read_csv(r"ExternalData\API_FR.INR.LEND_DS2_en_csv_v2_15559.csv",skip_rows=4).drop("Indicator Name","Indicator Name", "Country Code")
    credit=pl.read_csv(r"ExternalData\API_IC.CRD.INFO.XQ_DS2_en_csv_v2_18417.csv",skip_rows=4).drop("Indicator Name","Indicator Name", "Country Code")
    legal=pl.read_csv(r"ExternalData\API_IC.LGL.CRED.XQ_DS2_en_csv_v2_18437.csv",skip_rows=4).drop("Indicator Name","Indicator Name", "Country Code")
    export=pl.read_csv(r"ExternalData\API_NE.EXP.GNFS.ZS_DS2_en_csv_v2_13376.csv",skip_rows=4).drop("Indicator Name","Indicator Name", "Country Code")
    
    gdp = pl.read_excel(r"ExternalData\imf-dm-export-20250330_gdp_capita_ppi.xls")
    infl = pl.read_excel(r"ExternalData\imf-dm-export-20250330_infl.xls")
    
    dfs = [unempl, gdp, infl, ease_of_business, cost_business, r_d, formal_training, theft, market_cap, urban, lend, credit, legal, export]
    countries_mapping = {country.name: country.alpha_3 for country in pycountry.countries}
    year_cols = [str(year) for year in range(1980, 2024)]
    neighbors_json_path=r"ExternalData\neighbors_of_countries.json"
    processed_dfs = []
    
    # Iterate through CSV files (sorted to ensure a consistent order)
    for data in dfs:
        for col in data.columns:
            if col[0:2] in ["19","20"]:
                    # Convert column to string (Utf8) first
                    data = data.with_columns(pl.col(col).cast(pl.Utf8).alias(col))
                    
                    # Replace "no data" with null and cast to float
                    data = data.with_columns(
                        pl.when(pl.col(col) == "no data")
                          .then(pl.lit(None))
                          .when(pl.col(col) == "")
                          .then(pl.lit(None))
                          .otherwise(pl.col(col))
                          .cast(pl.Float64)
                          .alias(col)
                    ) 

                    data = data.with_columns(
                        pl.col(col)
                        .fill_null(pl.col(col).quantile(0.25))
                        .alias(col)
                    )
            
            first_col = data.columns[0]
            if first_col != "country":
                data = data.rename({first_col: "country"})
            
            data = data.with_columns(
                pl.col("country")
                  .str.extract(r"^([^,]+)")
                  .replace(countries_mapping)
                  .alias("country_code")
            )
        processed_dfs.append(data)
    
    if len(processed_dfs) < 3:
        raise ValueError("Expected at least three CSV files for unemployment, GDP, and inflation data.")
    
    # ========= STEP 3: Impute Missing External Data Using Bordering Countries =========
    with open(neighbors_json_path, "r") as f:
        borders_mapping = json.load(f)
    
    def get_alpha3(country_str: str) -> str:
        """Return the ISO alpha-3 code for a given country name or official name."""
        country_obj = pycountry.countries.get(name=country_str)
        if country_obj is None:
            country_obj = pycountry.countries.get(official_name=country_str)
        return country_obj.alpha_3 if country_obj is not None else None
    
    # Build a DataFrame of borders where each row is a mapping from a country to one of its neighbors.
    rows = []
    for country, neighbors in borders_mapping.items():
        main_code = get_alpha3(country)
        if main_code is None:
            continue
        for neighbor in neighbors.keys():
            neighbor_code = get_alpha3(neighbor)
            if neighbor_code is not None:
                rows.append({"country": main_code, "neighbor": neighbor_code})
    borders_df = pl.DataFrame(rows)
    
    def process_external(data: pl.DataFrame, value_col: str) -> pl.DataFrame:
        """
        Convert the external data (e.g. unemployment, GDP, inflation) to a long format,
        then join bordering country information to compute the mean of neighbor values to
        fill in missing data.
        """
        # Melt to long format
        data_long = data.melt(
            id_vars="country_code",
            value_vars=year_cols,
            variable_name="year",
            value_name=value_col
        ).with_columns(pl.col("year").cast(pl.Int64))
        
        # Join with borders_df to gather neighboring information
        neighbors_data = data_long.join(
            borders_df,
            left_on="country_code",
            right_on="neighbor",
            how="inner"
        )
        
        # For each main country and year, compute the mean of the value from its neighbors
        neighbors_mean = neighbors_data.group_by(["country", "year"]).agg(
            pl.col(value_col).mean().alias(f"{value_col}_mean")
        )
        
        # Join the computed neighbor means back to the long DataFrame and fill nulls
        data_filled = data_long.join(
            neighbors_mean,
            left_on=["country_code", "year"],
            right_on=["country", "year"],
            how="left"
        ).with_columns(
            pl.col(f"{value_col}_mean").cast(pl.Float64),
            pl.when(pl.col(value_col).is_null())
              .then(pl.col(f"{value_col}_mean"))
              .otherwise(pl.col(value_col)).cast(pl.Float64)
              .alias(value_col),
        )
        return data_filled

    dfs = [unempl, gdp, infl, ease_of_business, cost_business, r_d, formal_training, theft, market_cap, urban, lend, credit, legal, export]

    unempl_filled = process_external(processed_dfs[0], "unemployment_rate")
    gdp_filled   = process_external(processed_dfs[1], "gdp_per_capita")
    infl_filled  = process_external(processed_dfs[2], "inflation")
    ease_of_business_filled = process_external(processed_dfs[3], "ease_of_business")
    cost_business_filled = process_external(processed_dfs[4], "cost_business")
    r_d_filled = process_external(processed_dfs[5], "r_d")
    formal_training_filled = process_external(processed_dfs[6], "formal_training")
    theft_filled = process_external(processed_dfs[7], "theft") 
    market_cap_filled = process_external(processed_dfs[8], "market_cap")
    urban_filled = process_external(processed_dfs[9], "urban")
    lend_filled = process_external(processed_dfs[10], "lend")
    credit_filled = process_external(processed_dfs[11], "credit")
    legal_filled = process_external(processed_dfs[12], "legal")
    export_filled = process_external(processed_dfs[13], "export")

    themes =["unemployment_rate", "gdp_per_capita", "inflation", "ease_of_business", "cost_business", "r_d", "formal_training", "theft", "market_cap", "urban", "lend", "credit", "legal", "export"]    
    dfs_filled=[unempl_filled, gdp_filled, infl_filled, ease_of_business_filled, cost_business_filled, r_d_filled, formal_training_filled, theft_filled, market_cap_filled, urban_filled, lend_filled, credit_filled, legal_filled, export_filled]

    for j,df_filled in enumerate(dfs_filled):
        for i in ['foundation_year', 'first_funding_year', 'last_funding_year']:
            suffix = f"_{i.split('_')[0]}"
            
            df = df.join(
                df_filled,
                left_on=["country_code", i],
                right_on=["country_code", "year"],
                how="left",
                suffix=suffix
            ).with_columns(
                pl.col(f"{themes[j]}{suffix if i!='foundation_year' else ''}").fill_null(pl.col(f"{themes[j]}{suffix if i!='foundation_year' else ''}").quantile(0.25).over(["subregion"])).fill_null(pl.col(f"{themes[j]}{suffix if i!='foundation_year' else ''}").quantile(0.25)),
                pl.col(f"{themes[j]}_mean{suffix if i!='foundation_year' else ''}").fill_null(pl.col(f"{themes[j]}_mean{suffix if i!='foundation_year' else ''}").quantile(0.25).over(["subregion"])).fill_null(pl.col(f"{themes[j]}_mean{suffix if i!='foundation_year' else ''}").quantile(0.25)),
            )
    
    return df
